# DATA PREPARATION PIPELINE | PROJECT ID: 24

In [ ]:
import sys
!{sys.executable} -m pip install -U ydata-profiling[notebook]
!pip install jupyter-contrib-nbextensions
import pandas as pd
from ydata_profiling import ProfileReport
import numpy as np
import re
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.9/390.9 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.8/385.8 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 10.8 MB/s eta 0:00:00
   ━━━━

In [ ]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [ ]:
#Importing DATASET
file_path= 'https://raw.githubusercontent.com/RaffaeleRusso001/DataAndInformationQuality/refs/heads/main/SupportDataset/Comune-di-Milano-Pubblici-esercizi.csv'
try:
  DB = pd.read_csv(file_path, sep=';',encoding='utf-16')
except Exception as e:
    print(e)

In [ ]:
#Importing external DATASET (VIARIO) for further processing
file_path= 'https://raw.githubusercontent.com/RaffaeleRusso001/DataAndInformationQuality/refs/heads/main/SupportDataset/VIARIO_Milano.csv'
try:
  MI = pd.read_csv(file_path, sep=';',encoding='utf-8')
except Exception as e:
    print(e)

In [ ]:
#Importing external Dataset (CIVICI)
file_path_civici='https://raw.githubusercontent.com/RaffaeleRusso001/DataAndInformationQuality/refs/heads/main/SupportDataset/CIVICI_Milano.csv'
try:
  MI_CIVICI = pd.read_csv(file_path_civici, sep=';',encoding='utf-8')
except Exception as e:
    print(e)

In [ ]:
#Overview DB
DB

Tipo esercizio storico pe              Insegna  \
0                                NaN                  NaN   
1                                NaN                  NaN   
2                                NaN                  NaN   
3                                NaN                  NaN   
4                                NaN                  NaN   
...                              ...                  ...   
6899  wine,birr.,pub enot.,caff.,the           bar cherry   
6900  wine,birr.,pub enot.,caff.,the            la balusa   
6901  wine,birr.,pub enot.,caff.,the  la champagnerie sas   
6902  wine,birr.,pub enot.,caff.,the          old rooster   
6903  wine,birr.,pub enot.,caff.,the            starhotel   

                                             Ubicazione Tipo via  \
0     ALZ NAVIGLIO GRANDE N. 12 ; isolato:057; (z.d. 6)      ALZ   
1                    ALZ NAVIGLIO GRANDE N. 44 (z.d. 6)      ALZ   
2                    ALZ NAVIGLIO GRANDE N. 48 (z.d. 6)      ALZ   
3                     ALZ NAVIGLIO GRANDE N. 8 (z.d. 6)      ALZ   
4                    ALZ NAVIGLIO PAVESE N. 24 (z.d. 6)      ALZ   
...                                                 ...      ...   
6899  VLE DORIA ANDREA N. 12 ; isolato:031; accesso:...      VLE   
6900  VIA GARIGLIANO N. 5 ; isolato:277; accesso: ac...      VIA   
6901  VIA SOTTOCORNO PASQUALE N. 4 ; isolato:014; ac...      VIA   
6902  VIA CASTROVILLARI N. 23 ; isolato:150; accesso...      VIA   
6903        PZA FONTANA N. 3 ;pza fontana 003; (z.d. 1)      PZA   

          Descrizione via Civico  Codice via  ZD        Forma commercio  \
0         NAVIGLIO GRANDE     12        5144   6                    NaN   
1         NAVIGLIO GRANDE     44        5144   6                    NaN   
2         NAVIGLIO GRANDE     48        5144   6                    NaN   
3         NAVIGLIO GRANDE      8        5144   6                    NaN   
4         NAVIGLIO PAVESE     24        5161   6                    NaN   
...                   ...    ...         ...  ..                    ...   
6899         DORIA ANDREA     12        2230   2  solo somministrazione   
6900           GARIGLIANO      5        1134   9  solo somministrazione   
6901  SOTTOCORNO PASQUALE      4        3152   4  solo somministrazione   
6902        CASTROVILLARI     23        6299   7  solo somministrazione   
6903              FONTANA      3         308   1  solo somministrazione   

     Forma commercio prev Forma vendita  \
0                     NaN           NaN   
1                     NaN           NaN   
2                     NaN           NaN   
3                     NaN           NaN   
4                     NaN           NaN   
...                   ...           ...   
6899     somministrazione         misto   
6900     somministrazione         misto   
6901     somministrazione         misto   
6902     somministrazione         misto   
6903     somministrazione         misto   

                                     Settore storico pe  \
0     Ristorante, trattoria, osteria;Genere Merceol....   
1                             Bar gastronomici e simili   
2                             Bar gastronomici e simili   
3     BAR CAFF� E SIMILI;Ristorante, trattoria, osteria   
4                             Bar gastronomici e simili   
...                                                 ...   
6899                     Wine,birr.,pub enot.,caff.,the   
6900                     Wine,birr.,pub enot.,caff.,the   
6901       BAR CAFF� E SIMILI;Bar gastronomici e simili   
6902                     Wine,birr.,pub enot.,caff.,the   
6903                     Wine,birr.,pub enot.,caff.,the   

      Superficie somministrazione  
0                            83.0  
1                            26.0  
2                            58.0  
3                           101.0  
4                            51.0  
...                           ...  
6899                         59.0  
6900                         40.0  
6901      

# 1. **DATA EXPLORATION AND DATA PROFILING**

Exploration of the dataset **DB** through the creation of a detailed report using the `ydata-profiling` library. This tool automatically generates an interactive overview, offering key insights such as summary statistics, missing values, correlations, and distribution patterns, while identifying data quality issues and anomalies.

In [ ]:
#CREATING A PROFILE REPORT
profile = ProfileReport(DB, title="Profiling Report", explorative=True)
profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
DB.describe()

Codice via           ZD  Superficie somministrazione
count  6904.000000  6904.000000                  6825.000000
mean   3555.016367     4.573001                    85.996777
std    2238.982545     2.763538                    89.676674
min       1.000000     1.000000                     2.000000
25%    1510.000000     2.000000                    42.000000
50%    3129.000000     4.000000                    64.000000
75%    5294.000000     7.000000                   100.000000
max    7602.000000     9.000000                  2336.000000

# **2. DATA QUALITY ASSESSMENT**

In this section, we evaluate the **quality of the dataset** by computing essential metrics for each column:
- **Uniqueness**: Proportion of unique values in the column compared to the total number of rows.
- **Distinctness**: Ratio of distinct (unique) values to the count of non-missing entries.
- **Constancy**: Measures the dominance of the most frequent value within the column.
- **Completeness**: Proportion of non-missing (non-null) values in the column compared to the total number of rows.

These metrics provide a detailed understanding of the dataset's structure, variability, and potential data quality issues, laying the foundation for effective data cleaning and preprocessing.

In [ ]:
#return the number of tuples and columns of the data source
print("[Rows, Columns]", DB.shape)
print("\n\n")

columns = DB.columns
print("Columns: \n", columns)
print("\n\n\n")

# for each column we compute the: Uniqueness, Distinctness, Constancy and Completeness
for col in columns:
    print("DQ Assessment for "+col+":")

    print("Datatype: ", DB[col].dtypes)

    DISTINCT = (DB[col].nunique())
    COUNT = DB[col].count()
    ROWS = len(DB[col])
    UNIQUENESS = DISTINCT / ROWS
    print("UNIQUENESS: ", UNIQUENESS)

    DISTINCTNESS = DISTINCT / COUNT
    print("DISTINCTNESS: ", DISTINCTNESS)

    MAX_COUNTS = max(DB[col].value_counts())
    CONSTANCY = MAX_COUNTS / COUNT
    print("CONSTANCY: ", CONSTANCY)

    NOT_NULL = DB[col].notnull().sum()
    NULL = DB[col].isnull().sum()
    COMPLETENESS = NOT_NULL / ROWS
    print("COMPLETENESS: ", COMPLETENESS)

    print("\n\n")


[Rows, Columns] (6904, 13)



Columns: 
 Index(['Tipo esercizio storico pe', 'Insegna', 'Ubicazione', 'Tipo via',
       'Descrizione via', 'Civico', 'Codice via', 'ZD', 'Forma commercio',
       'Forma commercio prev', 'Forma vendita', 'Settore storico pe',
       'Superficie somministrazione'],
      dtype='object')




DQ Assessment for Tipo esercizio storico pe:
Datatype:  object
UNIQUENESS:  0.003331402085747393
DISTINCTNESS:  0.004143397586020537
CONSTANCY:  0.5898036389839668
COMPLETENESS:  0.8040266512166859



DQ Assessment for Insegna:
Datatype:  object
UNIQUENESS:  0.4174391657010429
DISTINCTNESS:  0.8248425872925015
CONSTANCY:  0.05838580423583286
COMPLETENESS:  0.5060834298957126



DQ Assessment for Ubicazione:
Datatype:  object
UNIQUENESS:  0.9157010428736964
DISTINCTNESS:  0.9157010428736964
CONSTANCY:  0.0010139049826187717
COMPLETENESS:  1.0



DQ Assessment for Tipo via:
Datatype:  object
UNIQUENESS:  0.002317497103128621
DISTINCTNESS:  0.002317497103128621
CONSTANCY

**DUPLICATION**


In [ ]:
DUPLICATES= DB.duplicated()
print("Duplicates: ", DUPLICATES.any())
print("Number of Duplicates: ", DB.duplicated().sum())

#show the % of duplicated tuples
percentage_duplicates= (DB.duplicated().sum()/len(DB))*100
print(f"Percentage of duplicated rows in DB: {percentage_duplicates:.2f}%")

print("\nDuplicated rows are: \n")
DB[DB.duplicated()]

Duplicates:  True
Number of Duplicates:  1
Percentage of duplicated rows in DB: 0.01%

Duplicated rows are: 



Tipo esercizio storico pe Insegna                  Ubicazione Tipo via  \
940                       NaN     NaN  VIA SALASCO N. 29 (z.d. 5)      VIA   

    Descrizione via Civico  Codice via  ZD Forma commercio  \
940         SALASCO     29        4051   5             NaN   

    Forma commercio prev Forma vendita           Settore storico pe  \
940                  NaN           NaN  Spaccio bevande analcoliche   

     Superficie somministrazione  
940                          NaN

**ACCURACY**

Evaluating the dataset's **accuracy** for columns related to the road system by leveraging an external dataset, assumed to be accurate, obtained from the [Milan Municipality's website](https://geoportale.comune.milano.it/sit/open-data-toponomastica/). This external reference is used to assess all aspects of the road network in Milan and compare them to the information available in our dataset.

In [ ]:
MI

CODICE_VIA  STATO  TIPO                      DENOMINAZIONE  \
0              1      2     3                        DUOMO (DEL)   
1             10      2    17                  VITTORIO EMANUELE   
2             96      2     1        CALDERON DE LA BARCA PIETRO   
3             97      2     4                   TOSCANINI ARTURO   
4             98      2     3                      LIBERTY (DEL)   
...          ...    ...   ...                                ...   
4404        9985      2    27                  POËT LIDIA (POET)   
4405        9986      2    27                  BASILICO GABRIELE   
4406        9987      2    27  SORELLE MIRABAL - "LAS MARIPOSAS"   
4407        9988      2    27                 FRANCESCHI ROBERTO   
4408        9989      2    27            FERRAZZUTTO BONAVENTURA   

      DATA_INTITOLAZIONE  ANNO_SOPPRESSIONE  \
0             17000101.0                  0   
1                    NaN                  0   
2             19721027.0                  0   
3                    NaN                  0   
4                    NaN                  0   
...                  ...                ...   
4404          20240223.0                  0   
4405          20231012.0                  0   
4406          20231123.0                  0   
4407          20240124.0                  0   
4408          20241002.0                  0   

                            DESCRITTIVO                             ANNCSU  \
0                             DEL DUOMO                          DEL DUOMO   
1                  VITTORIO EMANUELE II          VITTORIO EMANUELE SECONDO   
2           PIETRO CALDERON DE LA BARCA        PIETRO CALDERON DE LA BARCA   
3                      ARTURO TOSCANINI                   ARTURO TOSCANINI   
4                           DEL LIBERTY                        DEL LIBERTY   
...                                 ...                                ...   
4404                         LIDIA POËT                         LIDIA POET   
4405                  GABRIELE BASILICO                  GABRIELE BASILICO   
4406  SORELLE MIRABAL - "LAS MARIPOSAS"  SORELLE MIRABAL - "LAS MARIPOSAS"   
4407                 ROBERTO FRANCESCHI                 ROBERTO FRANCESCHI   
4408            BONAVENTURA FERRAZZUTTO            BONAVENTURA FERRAZZUTTO   

                          OPENSTREETMAP  PROGANNCSU  
0                             DEL DUOMO         NaN  
1             VITTORIO EMANUELE SECONDO         NaN  
2           PIETRO CALDERON DE LA BARCA         NaN  
3                      ARTURO TOSCANINI         NaN  
4                               LIBERTY         NaN  
...                                 ...         ...  
4404                         LIDIA POËT         NaN  
4405                  GABRIELE BASILICO         NaN  
4406  SORELLE MIRABAL - "LAS MARIPOSAS"         NaN  
4407                 ROBERTO FRANCESCHI         NaN  
4408            BONAVENTURA FERRAZZUTTO         NaN  

[4409 rows x 10 columns]

**Accuracy Codice Via**

In [ ]:
# Extract the relevant columns ("Descrizione via" and "Codice via") from the main dataset.
df1 = DB[["Descrizione via", "Codice via"]]
df2 = MI

# Merge the extracted dataset with the external reference dataset using the "Codice via" field.
df3 = pd.merge(df1, df2, left_on="Codice via", right_on="CODICE_VIA", how="left", suffixes=("_df1", "_df2"))

# Add a new column to check if the "Descrizione via" matches any corresponding description fields in the external dataset.
df3["descrizione_match"] = (df3["Descrizione via"] == df3["DESCRITTIVO"]) | (df3["Descrizione via"] == df3["DENOMINAZIONE"]) | (df3["Descrizione via"] == df3["ANNCSU"]) | (df3["Descrizione via"] == df3["OPENSTREETMAP"])

# Filter the merged DataFrame to include only rows where the match condition is False.
df_filtered = df3[df3["descrizione_match"] == False]

# Calculate the accuracy as the proportion of rows with a match compared to the total number of rows.
acc = 1 - (df_filtered.shape[0] / df1.shape[0])
print(f"Accuracy Denominazione via: {acc:.4f}")

Accuracy Denominazione via: 0.9709


**Accuracy Municipio**

Loading an external dataset containing civic information for Milan. We compare the address data from the main dataset (DB) with the external MI_CIVICI dataset to verify the accuracy of the "Municipio" (district) information. It first extracts the relevant columns, converts the "Civico" column to numeric, and then merges the two datasets based on "Codice via" and "Civico" (or "NUMERO" in the external dataset). A new column, municipio_match, is created to check if the "ZD" value matches the "MUNICIPIO" value from the external dataset. Finally, the code calculates and prints the accuracy of the "Municipio" match.

In [ ]:
df4 = DB[["Codice via", "Civico", "ZD"]]
df4.loc[:, "Civico"] = pd.to_numeric(df4["Civico"], errors="coerce")

df5 = MI_CIVICI[["CODICE_VIA", "NUMERO", "MUNICIPIO"]]

df6 = pd.merge(df4, df5, left_on=["Codice via", "Civico"], right_on=["CODICE_VIA", "NUMERO"], how="left", suffixes=("_df1", "_df2"))

df6["municipio_match"] = (df6["ZD"] == df6["MUNICIPIO"])

df_filtered6 = df6[df6["municipio_match"] == False]

acc = 1 - (df_filtered6.shape[0] / df4.shape[0])
print(f"Accuracy Municipio: {acc:.4f}")

Accuracy Municipio: 0.9742


**CONSISTENCY**

Considering the domain on which the dataset is built, we define the following rules:

- Check for the presence of negative numerical house numbers in the "Civico" column.
- Check for records where the same "Codice via" is associated with different "Descrizione via" values.


In [ ]:
# Group by 'Codice via' and 'Descrizione via' and count the occurrences."
via_counts = DB.groupby(["Codice via", "Descrizione via"]).size().reset_index(name="Count")
inconsistent_codici = via_counts.groupby("Codice via").filter(lambda x: len(x) > 1)
print(inconsistent_codici)

# Filtering on negative Civico
df7 = DB
df7["Civico"] = pd.to_numeric(df7["Civico"], errors="coerce")
civici_negativi = DB[DB["Civico"] < 0]
print(civici_negativi)

Empty DataFrame
Columns: [Codice via, Descrizione via, Count]
Index: []
Empty DataFrame
Columns: [Tipo esercizio storico pe, Insegna, Ubicazione, Tipo via, Descrizione via, Civico, Codice via, ZD, Forma commercio, Forma commercio prev, Forma vendita, Settore storico pe, Superficie somministrazione]
Index: []


# **3. DATA CLEANING (DATA TRANSFORMATION)**

In this section, we focus on the data transformation process, where we apply specific rules and techniques to clean, modify, and structure the dataset for further analysis.

In [ ]:
#Standardizes the column names by removing spaces and replacing them with underscores for easier reference and use in further analysis.
DB.columns = DB.columns.str.replace(' ', '_')
DB.columns

Index(['Tipo_esercizio_storico_pe', 'Insegna', 'Ubicazione', 'Tipo_via',
       'Descrizione_via', 'Civico', 'Codice_via', 'ZD', 'Forma_commercio',
       'Forma_commercio_prev', 'Forma_vendita', 'Settore_storico_pe',
       'Superficie_somministrazione'],
      dtype='object')

**Focus: "Tipo_esercizio_storico_pe"**





In [ ]:
# Wrangling [Tipo_esercizio_storico_pe]: changing the type of the columns Tipo_esercizio_storicoPe to string
DB['Tipo_esercizio_storico_pe'] = DB['Tipo_esercizio_storico_pe'].astype(str)
print("Tipo_esercizio_storico_pe: \n",DB.Tipo_esercizio_storico_pe.unique())

Tipo_esercizio_storico_pe: 
 ['nan' 'bar caff�' "bar-caffe' e simili" 'bar gastronomici e simili'
 'bar pasticc.gelat.crem.creper.' 'birreria' 'cibi cotti'
 'cibi cotti preconfezionati' 'gelateria' 'genere merceol.autorizz.sanit.'
 'mensa' 'osteria' 'pizzeria' 'pizzerie e simili'
 'prodotti di gastronomia' 'ristorante' 'ristorante, trattoria, osteria'
 'sale da ballo, locali notturni' 'spaccio bevande analcoliche'
 'tav.calde,self service,fast f.' 'tavola calda' 'tavola fredda'
 'trattoria' 'wine,birr.,pub enot.,caff.,the']


Performing data cleaning and standardization on the **Tipo_esercizio_storico_pe** column of the DB DataFrame, which is derived from the reference dataset of the road system of the Municipality of Milan. The goal is to normalize various types of business establishments into more consistent categories.

The code uses regular expressions (**regex**) and conditional statements to identify and update specific values in the column. The transformations include:

1.   Replacing specific patterns of strings (e.g., variations of "bar caffè") with a standard label, such as "bar caffetteria"."
2.   Correcting and standardizing certain business types, such as "pizzerie e simili" to "pizzeria e simili" or "wine,birr.,pub enot.,caff.,the" to "wine,birreria,pub,enoteca,caffetteria,the."

In [ ]:
pattern = r'bar caff.*'
DB.loc[DB['Tipo_esercizio_storico_pe'].str.contains(pattern), 'Tipo_esercizio_storico_pe'] = 'bar caffetteria'
pattern2=r'bar-caffe.*'
DB.loc[DB['Tipo_esercizio_storico_pe'].str.contains(pattern2), 'Tipo_esercizio_storico_pe'] = 'bar caffetteria e simili'
DB.loc[DB['Tipo_esercizio_storico_pe'] == 'bar pasticc.gelat.crem.creper.','Tipo_esercizio_storico_pe'] = 'bar,pasticceria,gelateria,cremeria,creperia'
DB.loc[DB['Tipo_esercizio_storico_pe'] == 'genere merceol.autorizz.sanit.','Tipo_esercizio_storico_pe'] = 'genere merceologico soggetto ad autorizzazione sanitaria'
DB.loc[DB['Tipo_esercizio_storico_pe'] == 'pizzerie e simili','Tipo_esercizio_storico_pe'] = 'pizzeria e simili'
DB.loc[DB['Tipo_esercizio_storico_pe'] == 'ristorante, trattoria, osteria','Tipo_esercizio_storico_pe'] = 'ristorante,trattoria,osteria'
DB.loc[DB['Tipo_esercizio_storico_pe'] == 'tav.calde,self service,fast f.','Tipo_esercizio_storico_pe'] = 'tavola calda,self service,fast food'
DB.loc[DB['Tipo_esercizio_storico_pe'] == 'wine,birr.,pub enot.,caff.,the','Tipo_esercizio_storico_pe'] = 'wine,birreria,pub,enoteca,caffetteria,the'

In [ ]:
print("Tipo_esercizio_storico_pe: \n",DB.Tipo_esercizio_storico_pe.unique())

Tipo_esercizio_storico_pe: 
 ['nan' 'bar caffetteria' 'bar caffetteria e simili'
 'bar gastronomici e simili' 'bar,pasticceria,gelateria,cremeria,creperia'
 'birreria' 'cibi cotti' 'cibi cotti preconfezionati' 'gelateria'
 'genere merceologico soggetto ad autorizzazione sanitaria' 'mensa'
 'osteria' 'pizzeria' 'pizzeria e simili' 'prodotti di gastronomia'
 'ristorante' 'ristorante,trattoria,osteria'
 'sale da ballo, locali notturni' 'spaccio bevande analcoliche'
 'tavola calda,self service,fast food' 'tavola calda' 'tavola fredda'
 'trattoria' 'wine,birreria,pub,enoteca,caffetteria,the']


**Focus: Ubicazione**

In this section we perform data extraction and transformation steps to clean and structure the `Ubicazione` column from the original dataset assuming that that the "Codice_via" column is "correct". The goal is to extract and standardize key components of the location information, creating new columns for specific attributes. The process is outlined as follows:

1. **Define Regular Expressions for Pattern Matching**:
   - Various patterns are defined to extract information such as (`Via`),  (`civico_ubicazione`), (`Isolato`), (`accesso`), (`ZD`), (`Ingresso`).

2. **Apply Regular Expressions to Extract Data**:
   - The `re.search()` function is used to extract the relevant data from the `Ubicazione` column based on the defined patterns. New columns are created for each type of extracted information, such as `via`, `civico_ubicazione`, and `isolato`.

3. **Clean and Standardize the Extracted Data**:
   - Additional cleaning steps are applied to standardize values, such as removing unnecessary text (e.g., "NUM.") or non-numeric characters, handling missing or invalid data, and formatting the extracted columns to ensure consistency.

4. **Remove Unwanted Data and Symbols**:
   - The `other` column is created to hold the remaining data after extracting the relevant parts (via, isolato, accesso, etc.). Regular expressions are used to remove any identified components from the `Ubicazione` column and clean up the remaining text, removing unnecessary symbols and whitespace.

5. **Final Cleanup**:
   - The column `Ubicazione` is dropped from the dataset as it is no longer needed after the transformation, leaving only the relevant, cleaned columns for further analysis.

In [ ]:
print("\n\nUbicazione:",DB.Ubicazione.unique())
#put all the values in the column Ubicazione uppercase letter
DB['Ubicazione'] = DB['Ubicazione'].str.upper()
print("\n\nUbicazione Upper case:",DB.Ubicazione.unique())



Ubicazione: ['ALZ NAVIGLIO GRANDE N. 12 ; isolato:057; (z.d. 6)'
 'ALZ NAVIGLIO GRANDE N. 44 (z.d. 6)' 'ALZ NAVIGLIO GRANDE N. 48 (z.d. 6)'
 ...
 'VIA SOTTOCORNO PASQUALE N. 4 ; isolato:014; accesso: accesso esterno; (z.d. 4)'
 'VIA CASTROVILLARI N. 23 ; isolato:150; accesso: accesso esterno; (z.d. 7)'
 'PZA FONTANA N. 3 ;pza fontana 003; (z.d. 1)']


Ubicazione Upper case: ['ALZ NAVIGLIO GRANDE N. 12 ; ISOLATO:057; (Z.D. 6)'
 'ALZ NAVIGLIO GRANDE N. 44 (Z.D. 6)' 'ALZ NAVIGLIO GRANDE N. 48 (Z.D. 6)'
 ...
 'VIA SOTTOCORNO PASQUALE N. 4 ; ISOLATO:014; ACCESSO: ACCESSO ESTERNO; (Z.D. 4)'
 'VIA CASTROVILLARI N. 23 ; ISOLATO:150; ACCESSO: ACCESSO ESTERNO; (Z.D. 7)'
 'PZA FONTANA N. 3 ;PZA FONTANA 003; (Z.D. 1)']


In [ ]:
# Define patterns for extraction
via_pattern = r"([A-Za-z\s'.]+?)\s*(?:N\.|NUM|NUMERO|num\.)?\s*(\d+(?:\/[A-Za-z\d]+(?:\/[A-Za-z\d]+)?)?)"
isolato_pattern = r"ISOLATO:(\d+)"
accesso_pattern = r"ACCESSO\s+DA|ACCESSO:\s([^;]+)"
zd_pattern = r"(Z\.D\.\s(\d+))"
ingresso_pattern = r"(?:CON\s+INGRESSO|CON\s+INGR\.?|CON\s+ING\.?|SU\s*INGR\.?|DA\s*INGR\.?|INGRESSO|INGR\.?|INGR\s*(?:DA|IN)|CON\s+INGR\s*(?:SU|DA|IN)|C/O|VERSO|ANGOLO|ANG.|INT\.\s*|INT\.|INTERNO\s*)\s*(.*?)(?:;|$)"

DB['via'] = DB['Ubicazione'].apply(lambda row: re.search(via_pattern, row).group(1) if re.search(via_pattern, row) else np.nan)
DB['civico_ubicazione'] = DB['Ubicazione'].apply(lambda row: re.search(via_pattern, row).group(2) if re.search(via_pattern, row) else np.nan)
DB['Isolato'] = DB['Ubicazione'].apply(lambda row: re.search(isolato_pattern, row).group(1) if re.search(isolato_pattern, row) else np.nan)
DB['accesso'] = DB['Ubicazione'].apply(lambda row: re.search(accesso_pattern, row).group(1) if re.search(accesso_pattern, row) else np.nan)
DB['ZD'] = DB['Ubicazione'].apply(lambda row: re.search(zd_pattern, row).group(1) if re.search(zd_pattern, row) else np.nan)
DB['Ingresso'] = DB['Ubicazione'].apply(lambda row: re.search(ingresso_pattern, row).group(1).strip() if re.search(ingresso_pattern, row) else np.nan)

DB['via'] = DB['via'].apply(lambda row: re.sub(r'\s*NUM\.\s*', '', row) if isinstance(row, str) else row)
DB['civico_ubicazione'] = DB['civico_ubicazione'].apply(lambda row: '0' if isinstance(row, str) and re.match(r'^0+$', row) else re.sub(r'[^0-9]', '', str(row)).lstrip('0') or '0' if pd.notnull(row) else row)
DB['ZD'] = DB['ZD'].str.extract(r'(\d+)', expand=False)
DB['Ingresso'] = DB['Ingresso'].astype(str)
DB['Ingresso'] = DB['Ingresso'].apply(lambda row: re.sub(r'\s*\(Z\.D\..*?\)', '', row))
DB['Ingresso'] = DB['Ingresso'].apply(lambda row: re.sub(r'\s*NUM\..*$', '', str(row)) if row is not np.nan else np.nan)
DB['Ingresso'] = DB['Ingresso'].apply(lambda row: re.sub(r'^(DA|SU|IN|ANCHE DA)\s+', '', row, flags=re.IGNORECASE))
DB['Isolato'] = DB['Isolato'].astype(str).apply(lambda row: re.sub(r'^0+', '', row) if row != 'nan' else row)

DB['other'] = DB['Ubicazione']
DB['other'] = DB['other'].apply(lambda row: re.sub(via_pattern, '', row))
DB['other'] = DB['other'].apply(lambda row: re.sub(isolato_pattern, '', row))
DB['other'] = DB['other'].apply(lambda row: re.sub(accesso_pattern, '', row))
DB['other'] = DB['other'].apply(lambda row: re.sub(zd_pattern, '', row))
DB['other'] = DB['other'].apply(lambda row: re.sub(ingresso_pattern, '', row))
DB['Civico'] = DB['Civico'].apply(lambda row: re.sub(r'[^0-9]', '', str(row).lstrip('0')) if pd.notnull(row) else row)
# Clean up 'other' column further by removing extra symbols and whitespace
DB['other'] = DB['other'].apply(lambda row: re.sub(r"[;:,\s()]+", ' ', row).strip())
DB['other'] = DB['other'].apply(lambda row: np.nan if not row or re.match(r"^[\s;:,.()]*$", row) else row)

# Dropping the column ubicazione
DB = DB.drop(columns=['Ubicazione'])

In [ ]:
# Splitting the 'via' column in DB to obtain 'Tipo_via_ubicazione' and 'Descrizione_via_ubicazione' obtained from the Ubicazione splitting
DB[['Tipo_via_ubicazione', 'Descrizione_via_ubicazione']] = DB['via'].str.split(n=1, expand=True)
#Dropping column via
DB = DB.drop(columns=['via'])

In [ ]:
#show the unique values
DB.Tipo_via_ubicazione.unique()

array(['ALZ', 'BST', 'CODVIA', 'CSO', 'GLL', 'LGO', 'PLE', 'PTA', 'PZA',
       'RIP', 'VIA', 'VLE', 'VIE', 'FOR', 'VLO', 'PAS', 'LARGO'],
      dtype=object)

In [ ]:
#Specifies the columns that should be set to NaN
columns_to_nan = [
    'civico_ubicazione', 'Isolato', 'accesso', 'Ingresso',
    'other', 'Tipo_via_ubicazione', 'Descrizione_via_ubicazione'
]

#Sets NaN only for the specified columns in rows where Tipo_via is 'CODVIA'
DB.loc[DB['Tipo_via_ubicazione'] == 'CODVIA', columns_to_nan] = pd.NA

#Checks the unique values in the Tipo_via column
print(DB.Tipo_via_ubicazione.unique())


['ALZ' 'BST' <NA> 'CSO' 'GLL' 'LGO' 'PLE' 'PTA' 'PZA' 'RIP' 'VIA' 'VLE'
 'VIE' 'FOR' 'VLO' 'PAS' 'LARGO']


This process helps clean the `Descrizione_via_ubicazione` column by removing any values that exist in the reference dataset `MI`, ensuring the data is more consistent for analysis.

In [ ]:
df12 = DB['Descrizione_via_ubicazione'].copy()

#Extract the comparison dataset (MI) with the columns to be used
df13 = MI[['DENOMINAZIONE', 'DESCRITTIVO', 'ANNCSU', 'OPENSTREETMAP']]

# Create a list of values to check against 'df12'
values_to_check = df13.values.flatten().tolist()

# Create a boolean mask using 'isin' to check if a value in df12 is in values_to_check
mask = df12.isin(values_to_check)

# Assign NaN to 'df12' where the mask is True
df12[mask] = np.nan

print(df12.unique())


[nan <NA> "PO'" "SANT' ALESSANDRO" "SANT' EUFEMIA" "SANT' EUSTORGIO"
 'DI PTA TICINESE' 'BARRILI  ANTON GIULIO' "COPERNICO NICOLO'"
 "D'APULIA NICOLA" "DELL' ALLODOLA" "DELL' INNOMINATO" "DELL' ORSO"
 "DELL' UNIONE" 'FIORAVANTI ARISTIDE' 'F.LLI ANTONA TRAVERSI'
 'F.LLI BRONZETTI' 'F.LLI FRASCHINI' 'F.LLI GRUBICY DE DRAGON'
 'F.LLI RUFFINI' "FRA' CRISTOFORO" 'MASSARA DE CAPITANI' "SANT' AGNESE"
 "SARPI FRA' PAOLO" 'GRAZIANO IMPERATORE' 'F.LLI CERVI' 'GOBETTI PIETRO'
 'IRNERIO CARLO' 'ARIOSTO LODOVICO' 'COLONNA MARCO ANTONIO'
 'DANTE ALIGHIERI' "DELL' ARCIVESCOVADO" 'F.LLI BRESSAN' 'F.LLI CAMPI'
 'F.LLI INDUNO' 'F.LLI POZZI' 'F.LLI ROSSELLI' 'F.LLI ZANZOTTERA'
 'F.LLI ZOIA' "FRA' GALGARIO" 'INAMA VIRGILIO' "SANT' ERLEMBALDO"
 "SANT' UGUZZONE" "SAULI SANT' ALESSANDRO" 'S. GIOV. SUL MURO'
 'TIZIANO VECELLIO' "DELL' ASSUNTA" 'MURILLO S.B.' 'F.LLI GABBA'
 'MILLY MIGNONE CARLA' "SANT' AGOSTINO" "DELL' INNOVAZIONE"
 'PELIZZA DA VOLPEDO' 'CAPPONI PIER LUIGI' 'F.LLI RIZZARDI'
 "DELL' ANNUNCIATA"

This section of the code focuses on cleaning and standardizing the values in the Tipo_via_ubicazione and Descrizione_via_ubicazione columns of the DB DataFrame.

In [ ]:
DB['Tipo_via_ubicazione'] = DB['Tipo_via_ubicazione'].str.replace('VIE', 'VIA')
DB['Tipo_via_ubicazione'] = DB['Tipo_via_ubicazione'].str.replace('LARGO', 'LGO')

articoli = ["L", "DELL", "ALL", "SULL", "NELL", "COL", "DALL", "D", "SANT"]
pattern = r'\b(' + '|'.join(articoli) + r")' (\w+)"
DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace(pattern, r"\1'\2", regex=True)

DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("F.LLI", "FRATELLI", regex=False)


DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("PO'", "PO")
DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("DI PTA TICINESE", "DI PORTA TICINESE")
DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("BARRILI  ANTON GIULIO", "BARRILI ANTON GIULIO")
DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("COPERNICO NICOLO'", "COPERNICO NICCOLO'")
DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("D'APULIA NICOLA", "NICOLA D'APULIA")
DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("FIORAVANTI ARISTIDE", "FIORAVANTI ARISTOTILE")
DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("FRATELLI ANTONA TRAVERSI", "FRATELLI CAMILLO E GIANNINO ANTONA TRAVERSI")
DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("FRA' CRISTOFORO", "FRA CRISTOFORO")
DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("MASSARA DE CAPITANI", "ROSA MASSARA DE CAPITANI")
DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("SARPI FRA' PAOLO", "SARPI PAOLO")
DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("GRAZIANO IMPERATORE", "GRAZIANO")
DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("GOBETTI PIETRO", "GOBETTI PIERO")
DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("IRNERIO CARLO", "IRNERIO")
DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("ARIOSTO LODOVICO", "ARIOSTO LUDOVICO")
DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("COLONNA MARCO ANTONIO", "COLONNA MARCANTONIO")
DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("DANTE ALIGHIERI", "DANTE")

DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("FRA' GALGARIO", "FRA GALGARIO")
DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("INAMA VIRGILIO", "INAMA VIGILIO")
DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("SAULI SANT'ALESSANDRO", "SANT'ALESSANDRO SAULI")
DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("S. GIOV. SUL MURO", "SAN GIOVANNI SUL MURO")
DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("TIZIANO VECELLIO", "TIZIANO")
DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("MURILLO S.B.", "MURILLO SEBASTIANO BARTOLOMEO")
DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("MILLY MIGNONE CARLA", "MIGNONE MILLY CARLA")
DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("PELIZZA DA VOLPEDO", "GIUSEPPE PELLIZZA DA VOLPEDO")
DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("CAPPONI PIER LUIGI", "CAPPONI PIERO")
DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("WINKELMANN GIOVANNI GIOACHINO", "GIOVANNI GIOACCHINO WINCKELMANN")
DB['Descrizione_via_ubicazione'] = DB['Descrizione_via_ubicazione'].str.replace("BATTISTI CESARE PARTE PRIVATA", "CESARE BATTISTI")

Same procedure applied on the Descrizione_via column

In [ ]:
articoli = ["L", "DELL", "ALL", "SULL", "NELL", "COL", "DALL", "D", "SANT"]
pattern = r'\b(' + '|'.join(articoli) + r")' (\w+)"
DB['Descrizione_via'] = DB['Descrizione_via'].str.replace(pattern, r"\1'\2", regex=True)

DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("F.LLI", "FRATELLI", regex=False)
DB['Tipo_via'] = DB['Tipo_via'].str.replace('VIE', 'VIA')
DB['Tipo_via'] = DB['Tipo_via'].str.replace('LARGO', 'LGO')

DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("PO'", "PO")
DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("DI PTA TICINESE", "DI PORTA TICINESE")
DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("BARRILI  ANTON GIULIO", "BARRILI ANTON GIULIO")
DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("COPERNICO NICOLO'", "COPERNICO NICCOLO'")
DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("D'APULIA NICOLA", "NICOLA D'APULIA")
DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("FIORAVANTI ARISTIDE", "FIORAVANTI ARISTOTILE")
DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("FRATELLI ANTONA TRAVERSI", "FRATELLI CAMILLO E GIANNINO ANTONA TRAVERSI")
DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("FRA' CRISTOFORO", "FRA CRISTOFORO")
DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("MASSARA DE CAPITANI", "ROSA MASSARA DE CAPITANI")
DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("SARPI FRA' PAOLO", "SARPI PAOLO")
DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("GRAZIANO IMPERATORE", "GRAZIANO")
DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("GOBETTI PIETRO", "GOBETTI PIERO")
DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("IRNERIO CARLO", "IRNERIO")
DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("ARIOSTO LODOVICO", "ARIOSTO LUDOVICO")
DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("COLONNA MARCO ANTONIO", "COLONNA MARCANTONIO")
DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("DANTE ALIGHIERI", "DANTE")

DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("FRA' GALGARIO", "FRA GALGARIO")
DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("INAMA VIRGILIO", "INAMA VIGILIO")
DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("SAULI SANT'ALESSANDRO", "SANT'ALESSANDRO SAULI")
DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("S. GIOV. SUL MURO", "SAN GIOVANNI SUL MURO")
DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("TIZIANO VECELLIO", "TIZIANO")
DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("MURILLO S.B.", "MURILLO SEBASTIANO BARTOLOMEO")
DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("MILLY MIGNONE CARLA", "MIGNONE MILLY CARLA")
DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("PELIZZA DA VOLPEDO", "GIUSEPPE PELLIZZA DA VOLPEDO")
DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("CAPPONI PIER LUIGI", "CAPPONI PIERO")
DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("WINKELMANN GIOVANNI GIOACHINO", "GIOVANNI GIOACCHINO WINCKELMANN")
DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("BATTISTI CESARE PARTE PRIVATA", "CESARE BATTISTI")

articoli = ["L", "DELL", "ALL", "SULL", "NELL", "COL", "DALL", "D", "SANT"]
pattern = r'\b(' + '|'.join(articoli) + r")' (\w+)"
DB['Descrizione_via'] = DB['Descrizione_via'].str.replace(pattern, r"\1'\2", regex=True)

DB['Descrizione_via'] = DB['Descrizione_via'].str.replace("F.LLI", "FRATELLI", regex=False)

In [ ]:
df19 = DB['Descrizione_via'].copy()

df21 = MI[['DENOMINAZIONE', 'DESCRITTIVO', 'ANNCSU', 'OPENSTREETMAP']]

values_to_check = df21.values.flatten().tolist()

mask = df19.isin(values_to_check)

df19[mask] = np.nan

print(df19.unique())

[nan "DELL'ARCIVESCOVADO"]


**Focus: Forma_commercio, Forma_commerco_prev**

Implementing a new mapping approach for handling unique values in the columns `Forma_commercio` and `Forma_commercio_prev`. For each unique value in these columns, a new boolean column is created where the corresponding row is marked as `True` if the value matches the current row in the original column, and `False` otherwise. If the value is `NaN`, it is mapped to `False`. After the mapping process, the original columns (`Forma_commercio` and `Forma_commercio_prev`) are dropped from the DataFrame if they are no longer needed.

Using binary representation of the unique values in each field, we are making easier analysis and filtering based on these specific categories.

In [ ]:
#'Forma_commercio'
for value in DB['Forma_commercio'].dropna().unique():
    column_name = f"Forma_commercio_{str(value).replace(' ', '_')}_boolean"
    DB[column_name] = DB['Forma_commercio'].apply(lambda x: x == value if pd.notna(x) else False)

#'Forma_commercio_prev'
for value in DB['Forma_commercio_prev'].dropna().unique():
    column_name = f"Forma_commercio_prev_{str(value).replace(' ', '_')}_boolean"
    DB[column_name] = DB['Forma_commercio_prev'].apply(lambda x: x == value if pd.notna(x) else False)

# Drop original column
DB = DB.drop(columns=['Forma_commercio', 'Forma_commercio_prev'])


**Focus: Settore storico pe**

In [ ]:
DB['Settore_storico_pe'] = DB['Settore_storico_pe'].astype('string')
DB['Settore_storico_pe'] = DB['Settore_storico_pe'].str.lower()

In this section, we implement a process to standardize the values in the `Settore_storico_pe` column of the DataFrame. A dictionary of search patterns and their corresponding replacements is used to update the values based on predefined rules. Each regular expression (regex) pattern in the dictionary is applied to search for matches within the strings in the column and replace them with a standardized value. The process is performed on each row, which may contain multiple parts, and the replacement is applied only to segments that match the pattern.

In [ ]:
pattern_dict = {
    r'bar caff.*': 'bar caffetteria',
    r'bar-caffe.*': 'bar caffetteria e simili',
    'bar pasticc.gelat.crem.creper.': 'bar,pasticceria,gelateria,cremeria,creperia',
    'genere merceol.autorizz.sanit.': 'genere merceologico soggetto ad autorizzazione sanitaria',
    'pizzerie e simili': 'pizzeria e simili',
    'ristorante, trattoria, osteria': 'ristorante,trattoria,osteria',
    'tav.calde,self service,fast f.': 'tavola calda,self service,fast food',
    'wine,birr.,pub enot.,caff.,the': 'wine,birreria,pub,enoteca,caffetteria,the',
    'altre apparecchiature elettr.':'altre apparecchiature elettriche',
    'ristorant':'ristorante',
    'disco-piano-americ.bar serali':'disco piano american bar serali',
    'tav.calde,self service,fast':'tavola calda,self service,fast food',
    'licenza giochi leciti/elettr..': 'licenza giochi leciti elettronici',
    'elettr.automatici meccaniche': 'elettronica,automatica,meccanica',
    'caff.* bar - somministrazione':'bar caffetteria,somministrazione',
    r"lett\. giochi leciti \+ societa['’]": "licenza giochi leciti tramite società"


}

for pattern, replacement in pattern_dict.items():
    DB['Settore_storico_pe'] = DB['Settore_storico_pe'].apply(
        lambda row: ';'.join(
            [
                replacement if re.search(pattern, part) else part
                for part in row.split(';')
            ]
        ) if isinstance(row, str) else row
    )



In [ ]:
#Define types for each and every column in DB
DB['Tipo_esercizio_storico_pe']=DB['Tipo_esercizio_storico_pe'].astype('string')
DB['Insegna']=DB['Insegna'].astype('string')
DB['Tipo_via']=DB['Tipo_via'].astype('string')
DB['Descrizione_via']=DB['Descrizione_via'].astype('string')
DB['Civico']=DB['Civico'].astype('string')
DB['Codice_via']=DB['Codice_via'].astype('string')
DB['ZD']=DB['ZD'].astype('int64')
DB['Forma_vendita']=DB['Forma_vendita'].astype('string')
DB['Settore_storico_pe']=DB['Settore_storico_pe'].astype('string')
DB['Codice_via']=DB['Codice_via'].astype('string')
DB['civico_ubicazione']=DB['civico_ubicazione'].astype('string')
DB['Isolato'] = pd.to_numeric(DB['Isolato'], errors='coerce')
DB['Isolato'] = DB['Isolato'].astype('Int64')
DB['accesso']=DB['accesso'].astype('string')
DB['Ingresso']=DB['Ingresso'].astype('string')
DB['Tipo_via_ubicazione']=DB['Tipo_via_ubicazione'].astype('string')
DB['Descrizione_via_ubicazione']=DB['Descrizione_via_ubicazione'].astype('string')

 We are updating the `Civico` column based on a condition where the values in the `Tipo_via` and `Descrizione_via` columns match the values in `Tipo_via_ubicazione` and `Descrizione_via_ubicazione`, respectively, and the `Civico` column is empty (null). When these conditions are met, the code assigns the value from the `civico_ubicazione` column to the `Civico` column. This helps to populate missing values in the `Civico` column using the data available in `civico_ubicazione` when the street type and description match.

In [ ]:
DB.loc[
    (DB['Tipo_via'] == DB['Tipo_via_ubicazione']) &
    (DB['Descrizione_via'] == DB['Descrizione_via_ubicazione']) &
    (DB['Civico'].isnull()),
    'Civico'
] = DB.loc[
    (DB['Tipo_via'] == DB['Tipo_via_ubicazione']) &
    (DB['Descrizione_via'] == DB['Descrizione_via_ubicazione']) &
    (DB['Civico'].isnull()),
    'civico_ubicazione'
].values

If the values in Tipo_via, Descrizione_via, and Civico do not match the corresponding values in Tipo_via_ubicazione, Descrizione_via_ubicazione, and civico_ubicazione, the columns isolato, accesso, and other are set to NaN (missing values). This ensures that the data in these columns is cleared when there is no match between the street type, description, and civic number. Finally, the code drops the columns Tipo_via_ubicazione, Descrizione_via_ubicazione, and civico_ubicazione from the DataFrame as they are no longer needed for further processing.

In [ ]:
DB.loc[
    (DB['Tipo_via'] != DB['Tipo_via_ubicazione']) &
    (DB['Descrizione_via'] != DB['Descrizione_via_ubicazione']) &
    (DB['Civico'] != DB['civico_ubicazione']),
    ['Isolato', 'accesso', 'other']
] = pd.NA

DB = DB.drop(columns=['Tipo_via_ubicazione', 'Descrizione_via_ubicazione', 'civico_ubicazione'])

In [ ]:
#replacing the 'nan' with NaN
DB['Tipo_esercizio_storico_pe'] = DB['Tipo_esercizio_storico_pe'].replace('nan', np.nan)
DB['Insegna'] = DB['Insegna'].replace('nan', np.nan)
DB['Isolato']=DB['Isolato'].replace('nan', np.nan)
DB['Ingresso']=DB['Ingresso'].replace('nan', np.nan)
DB['other']=DB['other'].replace('nan', np.nan)

This section of the code defines a function called `sort_and_deduplicate_sectors` which processes strings in the `Settore_storico_pe` column. The function performs two main tasks:

1. **Sorts Alphabetically**: It first splits the string based on the delimiter `;`, then sorts the resulting parts alphabetically.
2. **Removes Duplicates**: It uses a set to ensure that only unique values are retained.

If the value is `NaN` or null, it returns it as is without any modification.

The function is then applied to every value in the `Settore_storico_pe` column of the DataFrame `DB`. This ensures that all sector-related strings are alphabetically sorted and free of duplicates.

In [ ]:
def sort_and_deduplicate_sectors(sector_string):
    if pd.isna(sector_string):
        return sector_string
    parts = {x.strip() for x in sector_string.split(';')}
    return ';'.join(sorted(parts))

DB['Settore_storico_pe'] = DB['Settore_storico_pe'].apply(sort_and_deduplicate_sectors)
DB['Settore_storico_pe']=DB['Settore_storico_pe'].astype('string')

#**4. DATA CLEANING (ERROR DETECTION&CORRECTION MISSING VALUES)**

In this section, we refer to the data profiling and data quality assessment phase, where we analyze the dataset to understand its structure and identify any missing values. By looking at the distribution of values and the extent of missing data in each column, we can assess the data's overall quality. Based on this evaluation, we can choose the best imputation techniques to handle the missing values, such as using the mode, ML tecniques or default value depending on the type of data.

In [ ]:
#compute the number of NaN for each column in DB
DB.isnull().sum()

for col in DB.columns:
  print(col, DB[col].isnull().sum()/DB.shape[0]*100)

Tipo_esercizio_storico_pe 19.597334878331402
Insegna 49.39165701042874
Tipo_via 0.0
Descrizione_via 0.0
Civico 0.0
Codice_via 0.0
ZD 0.0
Forma_vendita 20.625724217844727
Settore_storico_pe 0.30417149478563155
Superficie_somministrazione 1.1442641946697567
Isolato 21.61066048667439
accesso 21.885863267670917
Ingresso 92.91714947856316
other 97.69698725376593
Forma_commercio_solo_somministrazione_boolean 0.0
Forma_commercio_somministrazione/minuto_boolean 0.0
Forma_commercio_prev_somministrazione_boolean 0.0
Forma_commercio_prev_minuto_boolean 0.0


Handling missing values in column Insegna, Using a default values in case it is missing, basically we assume that it is not specified

In [ ]:
#drop column other since it contains no significant additional informations
DB = DB.drop(columns=['other'])

The management of missing values in the "Insegna" (business name) column is challenging because business names are inherently personal and subjective. They depend on individual choices, branding, and other factors that do not follow a strict or standardized logic. As a result, it is difficult to apply a perfect rule to fill in these missing values. In this case, using a default value '**unspecified**' for missing entries is a practical solution. This approach ensures the dataset remains consistent and usable for analysis, while acknowledging that it cannot fully capture the unique nature of each business name.

In [ ]:
#fill the missing rows in column Insegna with the keyword unspecified
DB['Insegna'] = DB['Insegna'].fillna('unspecified')

 A new column, `Ingresso_Finale` is created by applying a condition that checks the values in the `ingresso` and `accesso` columns. If `ingresso` is not null, its value is used for `Ingresso_Finale`. If `ingresso` is null but `accesso` contains a value, `accesso` is used instead. If both columns are null, the default value "**ACCESSO ESTERNO**" is assigned. After the new column is created, the original `ingresso` and `accesso` columns are dropped, and `Ingresso_Finale` is renamed to `ingresso` for consistency in the dataset. This ensures that the final dataset has no missing values in the relevant column while maintaining clarity.

In [ ]:
DB['Ingresso_Finale'] = DB.apply(
    lambda row: row['Ingresso'] if pd.notnull(row['Ingresso'])  # Se Ingresso non è null, prendi Ingresso
    else (row['accesso'] if pd.notnull(row['accesso']) else 'ACCESSO ESTERNO'),  # Se Ingresso è null, ma Accesso non è null, prendi Accesso
    axis=1
)
DB.drop(columns=['Ingresso', 'accesso'], inplace=True)
DB.rename(columns={'Ingresso_Finale': 'Ingresso'}, inplace=True)
DB['Ingresso']=DB['Ingresso'].astype('string')

### Missing Values Management:

- **Numerical Variables:**
  - **Isolato**: Imputation performed using a **Random Forest Regressor**, which predicts the missing values based on other numerical features.
  - **Superficie_di_somministrazione**: The missing values are imputed using the **mode** (most frequent value) of the column, which is a straightforward and effective approach for categorical-like numerical data.

- **Categorical Variables:**
  - **Forma_vendita**: Missing values are imputed using a **Random Forest Classifier**, which leverages relationships with other columns to predict the missing categories.
  - **Tipo_esercizio_storico_pe**: Similarly, **Random Forest Classifier** is applied for imputing missing values in this column based on other features.
  - **Settore_storico_pe**: Since there are few missing values in this column, the **mode** is used for imputation, ensuring consistency with the overall dataset.

It's important to note that for columns like **Forma_vendita**, **Tipo_esercizio_storico_pe**, and others where mapping was applied, missing values were initially mapped to **False**. This mapping already ensures a 100% completeness for these columns, eliminating the need for further imputation.

In [ ]:
#Superficie_di_somministrazione
# Calculate the mode of 'Superficie_di_somministrazione'
mode_value = DB['Superficie_somministrazione'].mode()[0]

# Fill missing values with the calculated mode
DB.loc[DB['Superficie_somministrazione'].isnull(), 'Superficie_somministrazione'] = mode_value

In [ ]:
#replace NaN values in Settore_storico_pe with mode
mode_value = DB['Settore_storico_pe'].mode()[0]
DB['Settore_storico_pe'].fillna(mode_value, inplace=True)


<ipython-input-116-e98756049b9e>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  DB['Settore_storico_pe'].fillna(mode_value, inplace=True)


In [ ]:
DB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6904 entries, 0 to 6903
Data columns (total 16 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Tipo_esercizio_storico_pe                        5551 non-null   string 
 1   Insegna                                          6904 non-null   string 
 2   Tipo_via                                         6904 non-null   string 
 3   Descrizione_via                                  6904 non-null   string 
 4   Civico                                           6904 non-null   string 
 5   Codice_via                                       6904 non-null   string 
 6   ZD                                               6904 non-null   int64  
 7   Forma_vendita                                    5480 non-null   string 
 8   Settore_storico_pe                               6904 non-null   string 
 9   Superficie_somministrazione   

In this section, we implement a function to impute missing values in specific columns of the dataset using machine learning techniques. The function utilizes a Random Forest model, applying a regressor for numerical columns and a classifier for categorical columns. It handles one-hot encoding for categorical variables and updates the missing values directly in the dataset. We apply this function to impute missing data for the "isolato" and "Tipo_esercizio_storico_pe" columns, ensuring that the dataset is complete for further analysis.

In [ ]:
def impute_column_inplace(data, target_column, categorical=False):
    # Separate the data with and without missing values in the target column
    missing_mask = data[target_column].isnull()
    not_missing_mask = ~missing_mask

    # Predictors (all other columns)
    predictors = data.drop(columns=[target_column])

    # Encode categorical variables in the predictors
    predictors = pd.get_dummies(predictors, drop_first=True)

    # Separate features and target
    X_train, y_train = predictors[not_missing_mask], data.loc[not_missing_mask, target_column]
    X_missing = predictors[missing_mask]

    # If categorical, encode the target column
    if categorical:
        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        model = RandomForestClassifier(random_state=42, n_estimators=100)
    else:
        model = RandomForestRegressor(random_state=42, n_estimators=100)

    # Train the model
    model.fit(X_train, y_train)

    # Predict the missing values
    predictions = model.predict(X_missing)

    # Handle predictions based on the column type
    if target_column == "Isolato":
        # For 'isolato' (Int64), round and convert to Int64
        predictions = predictions.round().astype('int64')
    elif categorical:
        # For categorical columns, inverse transform the values with LabelEncoder
        predictions = le.inverse_transform(predictions)
        # Ensure predictions are of string type
        predictions = predictions.astype(str)

    # Update the missing values directly in the original dataset
    data.loc[missing_mask, target_column] = predictions


impute_column_inplace(DB, "Isolato", categorical=False)
impute_column_inplace(DB, "Tipo_esercizio_storico_pe", categorical=True)

# Verify missing values after imputation
missing_after_specific_imputation = DB.isnull().sum()
print(missing_after_specific_imputation)


Tipo_esercizio_storico_pe                             0
Insegna                                               0
Tipo_via                                              0
Descrizione_via                                       0
Civico                                                0
Codice_via                                            0
ZD                                                    0
Forma_vendita                                      1424
Settore_storico_pe                                    0
Superficie_somministrazione                           0
Isolato                                               0
Forma_commercio_solo_somministrazione_boolean         0
Forma_commercio_somministrazione/minuto_boolean       0
Forma_commercio_prev_somministrazione_boolean         0
Forma_commercio_prev_minuto_boolean                   0
Ingresso                                              0
dtype: int64


# Gestione Forma_vendita

In [ ]:
def impute_column(data, target_column, predictor_column, categorical=False):

    missing_mask = data[target_column].isnull()
    not_missing_mask = ~missing_mask
    predictors = data[[predictor_column]]

    X_train, y_train = predictors[not_missing_mask], data.loc[not_missing_mask, target_column]
    X_missing = predictors[missing_mask]


    if categorical:
        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        model = RandomForestClassifier(random_state=42, n_estimators=100)


    model.fit(X_train, y_train)


    predictions = model.predict(X_missing)

    if categorical:
        predictions = le.inverse_transform(predictions.astype(int))
        predictions = predictions.astype(str)

    data.loc[missing_mask, target_column] = predictions

impute_column(DB, "Forma_vendita", "Superficie_somministrazione", categorical=True)

missing_after_imputation = DB.isnull().sum()
print(missing_after_imputation)


Tipo_esercizio_storico_pe                          0
Insegna                                            0
Tipo_via                                           0
Descrizione_via                                    0
Civico                                             0
Codice_via                                         0
ZD                                                 0
Forma_vendita                                      0
Settore_storico_pe                                 0
Superficie_somministrazione                        0
Isolato                                            0
Forma_commercio_solo_somministrazione_boolean      0
Forma_commercio_somministrazione/minuto_boolean    0
Forma_commercio_prev_somministrazione_boolean      0
Forma_commercio_prev_minuto_boolean                0
Ingresso                                           0
dtype: int64


In [ ]:
#export to csv DB final after inputing
DB.to_csv('DB_after_inputing.csv', index=False)

# **5. DATA CLEANING (ERROR DETECTION&CORRECTION OUTLIERS)**

Focusing on handling missing values, particularly in light of the distribution of categorical versus numerical variables within the dataset. Given that categorical variables are more prevalent, our approach to analyse missing values takes this into account.

We begin by performing a frequency analysis on each column to understand the distribution of data. For numerical variables, we summarize their statistics, including quantiles, and apply the IQR method to detect potential outliers. For categorical variables, we calculate the frequency distribution of each category and identify rare categories with low frequencies (below 1%), which may indicate outliers.

In [ ]:
# Analyze frequency distribution for each column
for col in DB.columns:
    print(f"\nFrequency analysis for column: {col}")

    if pd.api.types.is_numeric_dtype(DB[col]) and not pd.api.types.is_bool_dtype(DB[col]):
        # Check if column is numeric and not boolean
        print("Numerical column summary:")
        print(DB[col].describe())

        # Calculate quantiles for detailed analysis
        quantiles = pd.to_numeric(DB[col], errors='coerce').quantile([0.01, 0.25, 0.5, 0.75, 0.99])
        print("\nQuantiles:")
        print(quantiles)

        # Identify potential outliers using the IQR method
        Q1 = quantiles[0.25]
        Q3 = quantiles[0.75]
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = DB[(DB[col] < lower_bound) | (DB[col] > upper_bound)]

        print("\nPotential outliers (IQR method):")
        if outliers.empty:
            print("No outliers detected.")
        else:
            print(outliers[col])

    elif not pd.api.types.is_numeric_dtype(DB[col]):  # Non-numeric data
        print("Categorical column summary:")
        value_counts = DB[col].value_counts(normalize=True) * 100
        print(value_counts)

        # Identify potential outliers (rare categories)
        threshold = 0.01  # Categories with less than 1% frequency
        rare_categories = value_counts[value_counts < threshold]
        print("\nPotential outliers (rare categories):")
        print(rare_categories)



Frequency analysis for column: Tipo_esercizio_storico_pe
Categorical column summary:
Tipo_esercizio_storico_pe
bar caffetteria                                             66.584589
ristorante                                                  11.993048
trattoria                                                    8.038818
pizzeria                                                     5.069525
tavola calda                                                 1.520857
spaccio bevande analcoliche                                  1.274623
ristorante,trattoria,osteria                                  0.89803
bar gastronomici e simili                                    0.709733
gelateria                                                    0.695249
osteria                                                      0.550406
tavola fredda                                                0.492468
bar caffetteria e simili                                     0.434531
genere merceologico soggetto ad autorizzazione s

#**6. DATA CLEANING (DATA DEDUPLICATION)**

---

### Record Linkage for Data Deduplication

In this section, we implement **Record Linkage** techniques for data deduplication, specifically focusing on identifying and removing duplicate records based on key attributes within the dataset.

The process begins by **creating an index for record linkage** using the `Codice_via` field as a blocking key. This step ensures that only records sharing the same `Codice_via` are compared, significantly reducing the number of comparisons. We use the `indexer.block('Codice_via')` method to create this index, generating candidate pairs for further comparison.

Next, we **define the comparison rules** to evaluate potential duplicates. We apply **exact matching** for columns like `Codice_via`, `Civico`, and `Tipo_esercizio_storico_pe`, ensuring that these values match exactly in the paired records. For the `Settore_storico_pe` column, we employ **string similarity** (using the Jaro-Winkler method) to account for slight text variations, with a threshold of 0.9 to determine a match.

The approach assumes that two public exercises are considered duplicates if they share the same street (`Codice_via`) and building number (`Civico`). Additional factors like the historical exercise type (`Tipo_esercizio_storico_pe`) and the historical sector (`Settore_storico_pe`) are also taken into account. However, cases where multiple public exercises exist within the same building (e.g., on different floors) are not treated as duplicates, even if they share the same street and building number.

Finally, we **compute the similarity scores** for each candidate pair based on the defined comparison rules. This results in a feature set that indicates the likelihood of two records being duplicates.

This approach ensures that we can accurately detect and remove duplicates, maintaining the quality and integrity of the dataset, while considering key assumptions about how records should be matched.

In [ ]:
!pip install recordlinkage
import recordlinkage

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.9/926.9 kB 10.2 MB/s eta 0:00:00


In [ ]:
DB.duplicated().any()
DB[DB.duplicated()]

Tipo_esercizio_storico_pe      Insegna Tipo_via Descrizione_via Civico  \
380              bar caffetteria  unspecified      VIA          CHIESE    600   
940  spaccio bevande analcoliche  unspecified      VIA         SALASCO    290   

    Codice_via  ZD Forma_vendita  \
380       1627   9         misto   
940       4051   5         misto   

                                    Settore_storico_pe  \
380  bar caffetteria;cibi cotti preconfezionati;gen...   
940                        spaccio bevande analcoliche   

     Superficie_somministrazione  Isolato  \
380                         90.0       84   
940                         60.0       84   

     Forma_commercio_solo_somministrazione_boolean  \
380                                          False   
940                                          False   

     Forma_commercio_somministrazione/minuto_boolean  \
380                                            False   
940                                            False   

     Forma_commercio_prev_somministrazione_boolean  \
380                                          False   
940                                          False   

     Forma_commercio_prev_minuto_boolean         Ingresso  
380                                False      VIALE SARCA  
940                                False  ACCESSO ESTERNO

In [ ]:
#Drop duplicated column
DB = DB.drop_duplicates()

# RecordLinkage

In [ ]:
# 1. Create an index for record linkage based on Codice_via
indexer = recordlinkage.Index()
indexer.block('Codice_via')

# 2. Create candidate comparisons
candidate_links = indexer.index(DB)

In [ ]:
# 3. Defining comparison rules
compare = recordlinkage.Compare()

compare.exact('Codice_via', 'Codice_via', label='Codice_via')
compare.exact('Civico', 'Civico', label='Civico')
compare.exact('Tipo_esercizio_storico_pe', 'Tipo_esercizio_storico_pe',label='Tipo_esercizio_storico_pe')
compare.string('Settore_storico_pe', 'Settore_storico_pe',method='jarowinkler', threshold=0.9,label='Settore_storico_pe')

features = compare.compute(candidate_links, DB)

In [ ]:
matches = features[features.sum(axis=1) >3]
print(len(matches))
matches

102


Codice_via  Civico  Tipo_esercizio_storico_pe  Settore_storico_pe
51   21             1       1                          1                 1.0
61   60             1       1                          1                 1.0
117  116            1       1                          1                 1.0
377  376            1       1                          1                 1.0
382  376            1       1                          1                 1.0
...               ...     ...                        ...                 ...
5571 5525           1       1                          1                 1.0
5764 5479           1       1                          1                 1.0
5878 5856           1       1                          1                 1.0
6290 6287           1       1                          1                 1.0
6448 6447           1       1                          1                 1.0

[102 rows x 4 columns]

In [ ]:
matches.index

MultiIndex([(  51,   21),
            (  61,   60),
            ( 117,  116),
            ( 377,  376),
            ( 382,  376),
            ( 382,  377),
            ( 383,  376),
            ( 383,  377),
            ( 383,  382),
            ( 426,  425),
            ...
            (4535, 2668),
            (4552, 3626),
            (4558, 1585),
            (4601, 1475),
            (5215, 4966),
            (5571, 5525),
            (5764, 5479),
            (5878, 5856),
            (6290, 6287),
            (6448, 6447)],
           length=102)

In [ ]:
# List to collect records in the desired format
rows = []

# Iterate through the found matches
for i in matches.index:
    # Get the original and candidate records
    record_1 = DB.iloc[i[0]].to_dict()  # Convert to dictionary
    record_2 = DB.iloc[i[1]].to_dict()  # Convert to dictionary

    # Add a "Type" column to distinguish original and candidate records
    record_1['Tipo'] = 'Originale'
    record_2['Tipo'] = 'Candidato'

    # Add the two records to the list
    rows.append(record_1)
    rows.append(record_2)

comparison_df = pd.DataFrame(rows)

comparison_df.to_csv('matches_comparison.csv', index=False)

In [ ]:
indexes = []
for i in matches.index:
    if i[1] not in indexes:
        DB = DB.drop([i[1]])
    indexes.append(i[1])

# Final result

Final reuslts : Data quality assessment

In [ ]:
#return the number of tuples and columns of the data source
print("[Rows, Columns]", DB.shape)
print("\n\n")

columns = DB.columns
print("Columns: \n", columns)
print("\n\n\n")

# for each column we compute the: Uniqueness, Distinctness, Constancy and Completeness
for col in columns:
    print("DQ Assessment for "+col+":")

    print("Datatype: ", DB[col].dtypes)

    DISTINCT = (DB[col].nunique())
    COUNT = DB[col].count()
    ROWS = len(DB[col])
    UNIQUENESS = DISTINCT / ROWS
    print("UNIQUENESS: ", UNIQUENESS)

    DISTINCTNESS = DISTINCT / COUNT
    print("DISTINCTNESS: ", DISTINCTNESS)

    MAX_COUNTS = max(DB[col].value_counts())
    CONSTANCY = MAX_COUNTS / COUNT
    print("CONSTANCY: ", CONSTANCY)

    NOT_NULL = DB[col].notnull().sum()
    NULL = DB[col].isnull().sum()
    COMPLETENESS = NOT_NULL / ROWS
    print("COMPLETENESS: ", COMPLETENESS)

    print("\n\n")


[Rows, Columns] (6811, 16)



Columns: 
 Index(['Tipo_esercizio_storico_pe', 'Insegna', 'Tipo_via', 'Descrizione_via',
       'Civico', 'Codice_via', 'ZD', 'Forma_vendita', 'Settore_storico_pe',
       'Superficie_somministrazione', 'Isolato',
       'Forma_commercio_solo_somministrazione_boolean',
       'Forma_commercio_somministrazione/minuto_boolean',
       'Forma_commercio_prev_somministrazione_boolean',
       'Forma_commercio_prev_minuto_boolean', 'Ingresso'],
      dtype='object')




DQ Assessment for Tipo_esercizio_storico_pe:
Datatype:  string
UNIQUENESS:  0.003376890324475114
DISTINCTNESS:  0.003376890324475114
CONSTANCY:  0.6623109675524886
COMPLETENESS:  1.0



DQ Assessment for Insegna:
Datatype:  string
UNIQUENESS:  0.42108354133020115
DISTINCTNESS:  0.42108354133020115
CONSTANCY:  0.4900895610042578
COMPLETENESS:  1.0



DQ Assessment for Tipo_via:
Datatype:  string
UNIQUENESS:  0.0020554984583761563
DISTINCTNESS:  0.0020554984583761563
CONSTANCY:  0.7374834826016737


In [ ]:
#export to csv DB
DB.to_csv('DB_final.csv', index=False)